<a href="https://colab.research.google.com/github/hanizarchmn/Tugas-DTI/blob/main/Classification_(Bank_Marketing_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   Nama   : Hanizar Rachman
*   NIP    : DS0113
*   Kelas  : Data Science

---



# Classification

Kali ini kita akan memodelkan model klasifikasi menggunakan dataset (https://www.kaggle.com/janiobachmann/bank-marketing-dataset).

Data Bank berisikan sebagai berikut :



1.   Age (umur) : angka (numeric)
2.   Job : Jenis pekerjaan (Categorical : 'admin.', 'blue-collar', 'entrepreneur' 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown')
3.   Marital : status pernikahan (Categorical : 'divorced', 'married', 'single', 'unknown' ; note: 'divorced' means divorced or widowed)
4.   Education (Categorical : 'Secondary', 'Tertiary', 'primary', 'unknown')
5.   Default : Mempunyai credit secara default? (Yes, No)
6.   Loan : Mempunyai personal loan? (Yes, No)
7.   Contact : Komunikasi menggunakan apa? (categorical: 'cellular','telephone', 'unknown')
8.   Day : Pada tanggal berapa terakhir berkomunikasi? (numeric)
9.   Month : Pada bulan apa terakhir kontak? (Categorical : 'may', 'aug', 'jul', 'jun', 'nov')
10.   Duration : Durasi kontak terakhir, dalam detik (numerik). Catatan penting: atribut ini sangat mempengaruhi target keluaran (misalnya, jika durasi = 0 maka y = 'tidak'). Namun, durasinya tidak diketahui sebelum panggilan dilakukan. Juga, setelah panggilan berakhir, y jelas diketahui. Dengan demikian, input ini hanya boleh dimasukkan untuk tujuan benchmark dan harus dibuang jika tujuannya adalah untuk memiliki model prediksi yang realistis.
11.   Campaign: Jumlah kontak yang dilakukan selama campaign ini dan untuk klien ini (numerik, termasuk kontak terakhir)
12.   Pdays : Jumlah hari yang berlalu setelah klien terakhir dihubungi dari campaign sebelumnya (numerik; 999 berarti klien sebelumnya tidak dihubungi)
13.   Previous : Jumlah kontak yang dilakukan sebelum campaign ini dan untuk klien ini (numerik)
14.   Housing : Mempunyai pinjam rumah? (Yes, No)
15.   Poutcome : Hasil dari *campaign* pemasaran sebelumnya (kategori: 'failure','nonexistent','success')



Import Library

In [3]:
#Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
#Import data ke Google Collabs
df_bank = pd.read_csv('https://raw.githubusercontent.com/faisalfaturohman/Clasisfication/main/bank.csv', sep = ',')
df_bank

# Menampilkan data
df_bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [4]:
#Cetak informasi data
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [5]:
#Cetak deskripsi statistika
df_bank.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,11162.0,41.231948,11.913369,18.0,32.0,39.0,49.00,95.0
balance,11162.0,1528.538524,3225.413326,-6847.0,122.0,550.0,1708.00,81204.0
day,11162.0,15.658036,8.420740,1.0,8.0,15.0,22.00,31.0
duration,11162.0,371.993818,347.128386,2.0,138.0,255.0,496.00,3881.0
campaign,11162.0,2.508421,2.722077,1.0,1.0,2.0,3.00,63.0
pdays,11162.0,51.330407,108.758282,-1.0,-1.0,-1.0,20.75,854.0
previous,11162.0,0.832557,2.292007,0.0,0.0,0.0,1.00,58.0


## Preporcessing Data

Mengecek data, apakah ada data yang hilang?

In [6]:
df_bank.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64